In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification

c:\Users\adamy\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_excel('datasetlabelled.xlsx')
df["label"] = df["label"].apply(lambda score: 1 if score == 1 else 0)
df['review'] =df['text']
df = df[["review", "label", ]]
# Get the underlying numpy arrays
reviews = df['review'].values
labels = df['label'].values

In [3]:
train_reviews, val_reviews, train_labels, val_labels = train_test_split(reviews, labels, test_size=.3)

In [4]:
checkpoint = "distilbert-base-uncased"
#Assign tokenizer object to the tokenizer class
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json: 100%|██████████| 48.0/48.0 [00:00<?, ?B/s]
c:\Users\adamy\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adamy\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config.json: 100%|███

In [5]:
def tokenize_dataset(reviews):
    encoded = tokenizer(
        reviews,
        padding=True,
        truncation=True,
        return_tensors='np',
    )
    return encoded.data
# Need to convert to List[str] because the tokenizer expects List but not np.array
tokenized_datasets = {
    "train": tokenize_dataset(train_reviews.tolist()),
    "validation": tokenize_dataset(val_reviews.tolist()),
}

In [6]:
batch_size = 8
num_epochs = 5
num_train_steps = (len(train_reviews) // batch_size) * num_epochs
# We let the declay goes from 1e-5 to 0 over course of training.

lr_scheduler = PolynomialDecay(
    initial_learning_rate=1e-5,
    end_learning_rate=0.,
    decay_steps=num_train_steps
)
# The optimizer is Adam with the learning rate schedule as specified
opt = Adam(learning_rate=lr_scheduler)

# Use the pretrained model from the same checkpoint as the tokenizer
# The num_label is 2 because we have a binary classification problem (Positive
# and negative)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# The model return logit (not probability), so we need to make sure to use the
# matching loss function to calculate the Cross Entropy from logits.
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# Compile the model and monitor the accuracy
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

model.safetensors: 100%|██████████| 268M/268M [00:19<00:00, 14.1MB/s] 


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [7]:
history = model.fit(
    tokenized_datasets['train'],
    train_labels,
    validation_data=(tokenized_datasets['validation'], val_labels),
    batch_size=batch_size,
    epochs=num_epochs
)
# END OF THE TRAINING PART FOR THE ALGORITHM IN CHARGE OF DETECTING THE SENTIMENT TOWARDS THE PRODUCT

Epoch 1/5


68/68 [==============================] - 78s 880ms/step - loss: 0.6148 - accuracy: 0.6865 - val_loss: 0.4132 - val_accuracy: 0.9221
Epoch 2/5
68/68 [==============================] - 54s 792ms/step - loss: 0.2587 - accuracy: 0.9295 - val_loss: 0.2185 - val_accuracy: 0.9221
Epoch 3/5
68/68 [==============================] - 53s 779ms/step - loss: 0.1383 - accuracy: 0.9629 - val_loss: 0.2102 - val_accuracy: 0.9307
Epoch 4/5
68/68 [==============================] - 51s 751ms/step - loss: 0.0870 - accuracy: 0.9852 - val_loss: 0.2382 - val_accuracy: 0.9177
Epoch 5/5
68/68 [==============================] - 50s 741ms/step - loss: 0.0644 - accuracy: 0.9944 - val_loss: 0.2088 - val_accuracy: 0.9264


In [9]:
Liste = ['I am deeply disappointed with this laptop',' I  like this laptop','I regret buying this laptop', "This computer is very practical for everyday tasks ",'After getting used to it, this computer is very pleasant to use.']
tokenized_inputs = tokenize_dataset(Liste)
tf_output = model.predict(tokenized_inputs)

tf_prediction = tf.nn.softmax(tf_output.logits, axis=1)
labels = ['Positive','Negative']
label = tf.argmax(tf_prediction, axis=1)
sentiment_list = []
for i in range(len(Liste)):
  if label[i] == 0:
    sentiment_list.append(labels[1])
  else:
    sentiment_list.append(labels[0])

print(sentiment_list)

1/1 [==============================] - 3s 3s/step
['Negative', 'Positive', 'Negative', 'Positive', 'Positive']
